In [1]:
#LIBRERIAS NECESARIAS

In [2]:

import requests
import numpy as np
import pandas as pd
from pandas import json_normalize
import os, time
import pickle, gzip
import json
import warnings
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline
warnings.filterwarnings("ignore", category=FutureWarning)


In [3]:
#REVISION APIS 

In [4]:
#REVISION API CATEGORIAS- 32 CATEGORIAS
cats = requests.get('https://api.mercadolibre.com/sites/MLA/categories')
cats.json()

[{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
 {'id': 'MLA1512', 'name': 'Agro'},
 {'id': 'MLA1403', 'name': 'Alimentos y Bebidas'},
 {'id': 'MLA1071', 'name': 'Animales y Mascotas'},
 {'id': 'MLA1367', 'name': 'Antigüedades y Colecciones'},
 {'id': 'MLA1368', 'name': 'Arte, Librería y Mercería'},
 {'id': 'MLA1743', 'name': 'Autos, Motos y Otros'},
 {'id': 'MLA1384', 'name': 'Bebés'},
 {'id': 'MLA1246', 'name': 'Belleza y Cuidado Personal'},
 {'id': 'MLA1039', 'name': 'Cámaras y Accesorios'},
 {'id': 'MLA1051', 'name': 'Celulares y Teléfonos'},
 {'id': 'MLA1648', 'name': 'Computación'},
 {'id': 'MLA1144', 'name': 'Consolas y Videojuegos'},
 {'id': 'MLA1500', 'name': 'Construcción'},
 {'id': 'MLA1276', 'name': 'Deportes y Fitness'},
 {'id': 'MLA5726', 'name': 'Electrodomésticos y Aires Ac.'},
 {'id': 'MLA1000', 'name': 'Electrónica, Audio y Video'},
 {'id': 'MLA2547', 'name': 'Entradas para Eventos'},
 {'id': 'MLA407134', 'name': 'Herramientas'},
 {'id': 'MLA1574', 'name': 'H

In [5]:
# Voy a enfocar mi ejercicio en la categoria de consolas y Videojuegos MLA1144
cats_2 = requests.get('https://api.mercadolibre.com/categories/MLA1144')
cats_2_1 = cats_2.json()
#cats_2_1

In [6]:
# AHORA QUIERO EXPLORAR LAS CATEGORIAS HIJAS (SUBCATEGORIAS-CONSOLAS)
subcategorias = cats_2_1.get('children_categories', [])
subcategorias_detalles = [(subcategory['id'], subcategory['name'], subcategory['total_items_in_this_category']) for subcategory in subcategorias]
subcategorias_detalles
# Encontramos 7 subcategorias (children_categories) dentro de la categoria 'Consolas y Videojuegos'

[('MLA438578', 'Accesorios para Consolas', 21708),
 ('MLA439527', 'Accesorios para PC Gaming', 812),
 ('MLA438566', 'Consolas', 72253),
 ('MLA8232', 'Flippers y Arcade', 2015),
 ('MLA438579', 'Repuestos para Consolas', 1777),
 ('MLA373840', 'Videojuegos', 53613),
 ('MLA438759', 'Otros', 900)]

In [7]:
# Ahora exploremos la subcategoria de mayor cantidad de elementos que es 'MLA438566', 'Consolas'
subcat = requests.get('https://api.mercadolibre.com/categories/MLA438566')
subcat_1 = subcat.json()
#subcat_1

In [8]:
# ES IMPORTANTE REVISAR Y ADENTARNOS EN LOS ATRIBUTOS 'MLA438566', 'Consolas'
atribut_subcat = requests.get('https://api.mercadolibre.com/categories/MLA438566/attributes')
atribut_subcat_1 = atribut_subcat.json()
atribut_subcat_1

[{'id': 'BRAND',
  'name': 'Marca',
  'tags': {'catalog_required': True,
   'required': True,
   'catalog_listing_required': True},
  'hierarchy': 'PARENT_PK',
  'relevance': 1,
  'value_type': 'string',
  'value_max_length': 255,
  'tooltip': 'Es el nombre de la empresa que elabora el producto.',
  'example': 'Sony',
  'attribute_group_id': 'OTHERS',
  'attribute_group_name': 'Otros',
  'hint': "Escribe la marca real del producto o 'Genérica' si no tiene marca."},
 {'id': 'LINE',
  'name': 'Línea',
  'tags': {'hidden': True},
  'hierarchy': 'PARENT_PK',
  'relevance': 1,
  'value_type': 'string',
  'value_max_length': 255,
  'attribute_group_id': 'OTHERS',
  'attribute_group_name': 'Otros'},
 {'id': 'MODEL',
  'name': 'Modelo',
  'tags': {'catalog_required': True, 'required': True},
  'hierarchy': 'PARENT_PK',
  'relevance': 1,
  'value_type': 'string',
  'value_max_length': 255,
  'tooltip': 'Es el nombre específico del producto.',
  'example': 'PlayStation',
  'attribute_group_id': 

In [9]:
# Vamos a entender cuales atributos son obligatorios en Consolas

# Realizamos la solicitud a la API
atribut_subcat = requests.get('https://api.mercadolibre.com/categories/MLA438566/attributes')
atribut_subcat_1 = atribut_subcat.json()

# Filtramos los nombres de los atributos que son obligatorios
atributos_obligatorios = [atributo['name'] for atributo in atribut_subcat_1 if atributo.get('tags', {}).get('required', False)]

# Mostramos los nombres de los atributos obligatorios
for atributo_nombre in atributos_obligatorios:
    print(atributo_nombre)
#ENTENDEMOS CUALES SON LOS ATRIBUTOS OBLIGATORIOS A DILIGENCIA POR SELLER EN EL MOENTO DE HACER LA PUBLICACIÓN

Marca
Modelo


In [10]:
#CREACION DE BASES DE INTERES

In [11]:
# POR EL MOMENTO SOLO HEMOS EXPLORADO DATA A NIVEL GENERAL CATEGORIAS SUBCATEGORIAS- Ahora vamos a explorar los items de la subcategoria Consolas MLA438566
# TENIENDO EN CEUNAT LA INFORMACIÓND DADA
cat_id = 'MLA438566'
offset = 0
url = f'https://api.mercadolibre.com/sites/MLA/search?category={cat_id}&offset={offset}'
request = requests.get(url)
# Aplanar la estructura anidada del JSON
definir = request.json()['results']
# Convertir a DataFrame
Item_1_plano = pd.DataFrame(definir)
# Exportar a Excel
Item_1_plano.to_excel('datos_mercadolibre_50.xlsx', index=False)


In [12]:
# teniendo en cuenta la consideraciones dadas en el PDF dado , ahora quiero hacerlo incrementando el offset (iteraciones de 50 en 50)
cat_id = 'MLA438566'

# Inicializar una lista para almacenar los DataFrames resultantes
items_dfs = []

# Iterar sobre los diferentes valores de offset
for offset in range(0, 1050, 50):
    # Construir la URL con el offset actual
    url = f'https://api.mercadolibre.com/sites/MLA/search?category={cat_id}&offset={offset}'
    
    # Realizar la solicitud a la API
    request = requests.get(url)
    
    # Verificar si la solicitud fue exitosa
    if request.status_code == 200:
        # Aplanar la estructura anidada del JSON
        comprimir = request.json()['results']
        
        # Convertir a DataFrame y agregarlo a la lista
        items_dfs.append(pd.DataFrame(comprimir))
    else:
        print(f"Error al obtener datos para el offset {offset}")

# Concatenar todos los DataFrames en uno solo
Item_1_plano_final = pd.concat(items_dfs, ignore_index=True)

# Visualizar el DataFrame resultante
print(Item_1_plano_final)




                 id                                              title  \
0     MLA1513384520  Sony Playstation 5 825gb God Of War Ragnarok B...   
1     MLA1437406762  Sony Playstation 4 Slim Cuh-20 1tb Standard  C...   
2     MLA1571014910  Consola Playstation Ps4 1tb God Of War Ragnaro...   
3     MLA1145652997  Nintendo Switch Oled 64gb Standard  Color Rojo...   
4     MLA1611398250  Consola Nintendo Switch Oled The Legend Of Zel...   
...             ...                                                ...   
1018  MLA1675176728  Nintendo Switch Usado Sin Caja Con Estuche Doc...   
1019  MLA1687615652                                            Play 3    
1020  MLA1697187058                      Ps4 Slim 1 Tb (caja Original)   
1021  MLA1711110546                                         Ps4 500 Gb   
1022  MLA1723216468                   Ps4 Edición Limitada Uncharted 4   

     condition                  thumbnail_id catalog_product_id  \
0          new  700033-MLA69689802995_052023

In [13]:
#dimensiones de nuestra base de datos 
Item_1_plano_final.shape

(1023, 35)

In [14]:
# Exportar a Excel (base de 1020 publiacioes de la subtegoria Consolas pertenecientea la categoria Consolas y video Juegos)
Item_1_plano_final.to_excel('datos_mercadolibre.xlsx', index=False)

In [15]:
# veo que es necesario abrir la informacióndel seller y los atributos por la manera en que vienen estructurados en la base original

Item_1_plano_final_aju = pd.DataFrame(Item_1_plano_final)

In [16]:
Nuevo0 = Item_1_plano_final_aju[['id','title', 'listing_type_id', 'price', 'original_price']]

In [17]:
# Crear Nuevo1 seleccionando las columnas 'id' y 'seller'
Nuevo1 = Item_1_plano_final_aju[['id', 'seller']]

nuevos_nombres = {'id': 'idd', 'seller': 'seller'}
Nuevo1 = Nuevo1.rename(columns=nuevos_nombres)

# Desagregar la columna 'seller' en columnas individuales
seller_df = json_normalize(Nuevo1['seller'])

# Copiar el DataFrame para evitar SettingWithCopyWarning
Nuevo1 = Nuevo1.copy()

# Eliminar la columna 'seller' original
Nuevo1.drop(columns=['seller'], inplace=True)

# Concatenar las columnas desagregadas con el DataFrame original
Nuevo1 = pd.concat([Nuevo1, seller_df], axis=1)

# Cambiar los nombres de las columnas
nuevos_nombres = {'idd': 'id', 'id': 'id_seller', 'nickname': 'nickname'}
Nuevo1 = Nuevo1.rename(columns=nuevos_nombres)

# Visualizar el DataFrame Nuevo1
print(Nuevo1.head())

              id    id_seller             nickname
0  MLA1513384520  576310358.0                S2GTO
1  MLA1437406762  267299678.0       SETEC BROTHERS
2  MLA1571014910   91570351.0          PHI-DIGITAL
3  MLA1145652997  758521932.0        PLAYTOGETHERS
4  MLA1611398250  190781661.0  BAIRES__ELECTRONICS


In [18]:
# Ejemplo de DataFrame Nuevo2
Nuevo2 = Item_1_plano_final_aju[['id', 'attributes']]  # DataFrame con las columnas 'id' y 'attributes'

# Inicializar un DataFrame vacío para almacenar los atributos expandidos
df_expandido = pd.DataFrame()

# Obtener una lista de todos los atributos únicos presentes en los datos
atributos_unicos = set()
for lista_de_diccionarios in Nuevo2['attributes']:
    for diccionario in lista_de_diccionarios:
        atributos_unicos.add(diccionario['name'])

# Crear columnas para cada atributo único
for atributo in atributos_unicos:
    df_expandido[atributo] = None

# Llenar el DataFrame expandido con los valores de los atributos
for i, lista_de_diccionarios in enumerate(Nuevo2['attributes']):
    for diccionario in lista_de_diccionarios:
        df_expandido.loc[i, diccionario['name']] = diccionario['value_name']

# Agregar la columna 'id' al DataFrame expandido
df_expandido['id'] = Nuevo2['id'].values

# renombrar
Nuevo2 = df_expandido

# Visualizar el DataFrame Nuevo1
print(Nuevo2.head())


  Código universal de producto      Peso Formato de venta Submodelo  \
0                          NaN    4.5 kg              NaN       NaN   
1                 000234885183    2.1 kg              NaN      Slim   
2                 711719560470      2 kg              NaN      Slim   
3                 045496883409  426.38 g              NaN      OLED   
4                4902370550481  426.38 g              NaN      OLED   

  Unidades por envase Largo del paquete Modelo detallado Es online  \
0                 NaN               NaN     CFI-1214A01X       NaN   
1                 NaN               NaN              NaN       NaN   
2                 NaN               NaN    Playstation 4       NaN   
3                 NaN               NaN      Switch OLED       NaN   
4                 NaN               NaN              NaN       NaN   

  Peso del paquete Conservación del producto Condición del ítem  \
0              NaN                       NaN              Nuevo   
1              NaN

In [19]:
# Unir los DataFrames Nuevo0, Nuevo1 
Items = pd.merge(Nuevo0, Nuevo1, on='id', how='inner')

In [20]:
# Unir los DataFrames Nuevo1, Nuevo2 
Items = pd.merge(Items, Nuevo2, on='id', how='inner')

In [21]:
Items.head(5)

,id,title,listing_type_id,price,original_price,id_seller,nickname,Código universal de producto,Peso,Formato de venta,...,Conservación del producto,Condición del ítem,Modelo,Modelo alfanumérico,Unidades por pack,Es offline,Color principal,Color,Marca,Línea
0,MLA1513384520,Sony Playstation 5 825gb God Of War Ragnarok B...,gold_pro,1199999.0,1355999.0,576310358.0,S2GTO,NaN,4.5 kg,NaN,...,NaN,Nuevo,PlayStation 5,NaN,NaN,NaN,Blanco,Blanco/Negro,Sony,PlayStation
1,MLA1437406762,Sony Playstation 4 Slim Cuh-20 1tb Standard C...,gold_special,580000.0,NaN,267299678.0,SETEC BROTHERS,000234885183,2.1 kg,NaN,...,NaN,Nuevo,PlayStation 4,CUH-20,NaN,NaN,Negro,Negro azabache,Sony,PlayStation
2,MLA1571014910,Consola Playstation Ps4 1tb God Of War Ragnaro...,gold_pro,799999.0,2267299.0,91570351.0,PHI-DIGITAL,711719560470,2 kg,NaN,...,NaN,Nuevo,PlayStation 4,NaN,NaN,NaN,Negro,Negro azabache,Sony,PlayStation
3,MLA1145652997,Nintendo Switch Oled 64gb Standard Color Rojo...,gold_special,514999.0,NaN,758521932.0,PLAYTOGETHERS,045496883409,426.38 g,NaN,...,NaN,Nuevo,Switch,NaN,NaN,NaN,Negro,Rojo neón/Azul neón/Negro,Nintendo,NaN
4,MLA1611398250,Consola Nintendo Switch Oled The Legend Of Zel...,gold_special,520000.0,NaN,190781661.0,BAIRES__ELECTRONICS,4902370550481,426.38 g,NaN,...,NaN,Nuevo,Nintendo Switch,NaN,NaN,NaN,Dorado,Dorado,Nintendo,NaN


In [22]:
Items.shape

(1023, 26)

In [23]:
# Exportar a Excel
Items.to_excel('datos_mercadolibre_items_api.xlsx', index=False)

In [24]:
#NUEVA DATA CON LOS CARACTERISTICAS DE LOS ITEMS

In [25]:
#aca consulo la informaciond e los items pero curzando con la información anteriormente obtenida

data = []

for item_id in Items['id']:
    # URL para obtener detalles del producto
    url = f'https://api.mercadolibre.com/items/{item_id}'

    # Realizar la solicitud a la API para obtener detalles del producto
    response = requests.get(url)

    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Convertir la respuesta JSON en un diccionario de Python
        product_details = response.json()

        # Agregar los detalles del producto al listado de datos
        data.append(product_details)
    else:
        print(f"Error al obtener detalles del producto {item_id}. Código de estado: {response.status_code}")

# Convertir el listado de datos en un DataFrame y cambiar el nombre del DataFrame a 'Items_full'
Items_full = pd.DataFrame(data)

# Visualizar el DataFrame resultante
print(Items_full)



                 id site_id  \
0     MLA1513384520     MLA   
1     MLA1437406762     MLA   
2     MLA1571014910     MLA   
3     MLA1145652997     MLA   
4     MLA1611398250     MLA   
...             ...     ...   
1018  MLA1675176728     MLA   
1019  MLA1687615652     MLA   
1020  MLA1697187058     MLA   
1021  MLA1711110546     MLA   
1022  MLA1723216468     MLA   

                                                  title   seller_id  \
0     Sony Playstation 5 825gb God Of War Ragnarok B...   576310358   
1     Sony Playstation 4 Slim Cuh-20 1tb Standard  C...   267299678   
2     Consola Playstation Ps4 1tb God Of War Ragnaro...    91570351   
3     Nintendo Switch Oled 64gb Standard  Color Rojo...   758521932   
4     Consola Nintendo Switch Oled The Legend Of Zel...   190781661   
...                                                 ...         ...   
1018  Nintendo Switch Usado Sin Caja Con Estuche Doc...   517268640   
1019                                            Play 3   10

In [26]:
# Exportar a Excel
Items_full.to_excel('datos_mercadolibre_items_full.xlsx', index=False)

In [27]:
#aca dejo la base que considero pertiente para poder obserar los items parecidos o iguales
# Seleccionar variable 'id' y variable 'attributes' para construir nuevo DF
Items_full2 = Items_full[['id', 'title', 'seller_id', 'category_id',
                          'price', 'base_price', 'original_price', 'listing_type_id', 'condition', 
                          'permalink', 'warranty', 'attributes']]

In [28]:
#vamos a obtener una base de datos con la información que considero es relevante

# Inicializar un DataFrame vacío para almacenar los atributos expandidos
df_expandido = pd.DataFrame()

# Obtener una lista de todos los atributos únicos presentes en los datos
atributos_unicos = set()
for lista_de_diccionarios in Items_full2['attributes']:
    if lista_de_diccionarios is not None:  # Comprobación para evitar errores si la lista está vacía
        for diccionario in lista_de_diccionarios:
            atributos_unicos.add(diccionario['name'])

# Crear un DataFrame temporal para almacenar los valores de los atributos
temp_df = pd.DataFrame(index=range(len(Items_full2)), columns=list(atributos_unicos))


# Llenar el DataFrame temporal con los valores de los atributos
for i, lista_de_diccionarios in enumerate(Items_full2['attributes']):
    if lista_de_diccionarios is not None:  # Comprobación para evitar errores si la lista está vacía
        for diccionario in lista_de_diccionarios:
            temp_df.at[i, diccionario['name']] = diccionario['value_name']

# Concatenar el DataFrame temporal con el DataFrame original
df_expandido = pd.concat([Items_full2, temp_df], axis=1)

# Eliminar la columna 'attributes' del DataFrame expandido
df_expandido.drop('attributes', axis=1, inplace=True)
Items_full_final = df_expandido

#aca quiero dejar  una variable de titulos sin espacios, uniendola para poder hacer comparaciones y adjuntandola a nuestra base de datos 
# Crear la variable 'title_aju' sin espacios ( con el fin de ealuar posible duplicados y mineria )
Items_full_final['title_aju'] = Items_full_final['title'].str.lower().str.replace(' ', '')

# Visualizar el DataFrame final
print(Items_full_final.head())

              id                                              title  \
0  MLA1513384520  Sony Playstation 5 825gb God Of War Ragnarok B...   
1  MLA1437406762  Sony Playstation 4 Slim Cuh-20 1tb Standard  C...   
2  MLA1571014910  Consola Playstation Ps4 1tb God Of War Ragnaro...   
3  MLA1145652997  Nintendo Switch Oled 64gb Standard  Color Rojo...   
4  MLA1611398250  Consola Nintendo Switch Oled The Legend Of Zel...   

   seller_id category_id      price  base_price  original_price  \
0  576310358   MLA438566  1199999.0   1199999.0       1355999.0   
1  267299678   MLA438566   580000.0    580000.0             NaN   
2   91570351   MLA438566   799999.0    799999.0       2267299.0   
3  758521932   MLA438566   514999.0    514999.0             NaN   
4  190781661   MLA438566   520000.0    520000.0             NaN   

  listing_type_id condition  \
0        gold_pro       new   
1    gold_special       new   
2        gold_pro       new   
3    gold_special       new   
4    gold_speci

In [29]:
Items_full_final.shape

(1023, 155)

In [30]:
# Exportar a Excel
Items_full_final.to_excel('datos_mercadolibre_items_full_final.xlsx', index=False)

In [31]:
#aca identifico que con estas  tres variables pudeo identificar items o publciaciones similares Titulo, Marca y Modelo.
# Obtener la tabla de frecuencias de las columnas 'title_aju', 'Modelo' y 'Marca'
frecuencias = Items_full_final[['title_aju', 'Modelo', 'Marca']].value_counts().reset_index()
frecuencias.columns = ['title_aju', 'Modelo', 'Marca', 'Frecuencia']

# Filtrar para mostrar solo las filas con frecuencia mayor a 1, esto con el fin de evaluar cuales son los items que estan duplcaidos 
frecuencias_filtradas = frecuencias[frecuencias['Frecuencia'] > 1 ]

# Mostrar la tabla de frecuencias filtrada
print(frecuencias_filtradas)

# Guardar la tabla de frecuencias filtrada en un archivo Excel(evaluar similaridades)
frecuencias_filtradas.to_excel('tabla_frecuencias.xlsx', index=False)

                                            title_aju  \
0                                        playstation4   
1                                        playstation5   
2                                             xbox360   
3   sonyplaystation4slim500gbstandardcolornegroaza...   
4                                        playstation4   
5          juegoconsolabrickgametetrisportatil9999en1   
6                                        playstation4   
7                                        playstation5   
8      sonyplaystation3superslim500gbconmandosycámara   
9                              sonyplaystation4pro1tb   
10                                       playstation5   
11                                                ps4   
12  playstation3superslimgarantía+joystick+juegofí...   
13                                   playstation4slim   
14       consolanetmaktetrisnm-brickretrovariosjuegos   
15        playstaion5-ps5cfi1215+godofwarragnaröksony   
16    sonyplaystation4500gbstan

In [32]:
#CONSTRUCCION BASE DE DATOS CON LAS PUBLICACIONES SIMILARES O IGUALES

In [33]:
#QUIERO CREAR DATAFRAMES CON CADA UNO DE LOS SEGMENTOS HALLADOS A TRAVÉZ DE LA EXPLORACION CON LAS TRESVARIABLES CONSIDERADAS

# Crear un diccionario para almacenar los DataFrames con los nombres modificados
dataframes_modificados = {}

# Iterar sobre las filas de la tabla de frecuencias filtradas
for index, row in frecuencias_filtradas.iterrows():
    title_aju = row['title_aju']
    modelo = row['Modelo']
    marca = row['Marca']
    
    # Filtrar el DataFrame original para obtener los registros que coinciden con los valores
    dataframe_filtrado = Items_full_final[(Items_full_final['title_aju'] == title_aju) & 
                                           (Items_full_final['Modelo'] == modelo) & 
                                           (Items_full_final['Marca'] == marca)]
    
    # Crear el nuevo nombre concatenando Marca, Modelo y title_aju con un guion bajo
    nuevo_nombre = f"{marca}_{modelo}_{title_aju}"
    
    # Agregar el DataFrame con el nuevo nombre al diccionario
    dataframes_modificados[nuevo_nombre] = dataframe_filtrado

# Mostrar los DataFrames con los nuevos nombres y también guardarlos en variables locales
for nombre, dataframe in dataframes_modificados.items():
    print(f"DataFrame '{nombre}':")
    print(dataframe)
    print('\n')
    # Guardar el DataFrame en una variable local con el mismo nombre
    locals()[nombre] = dataframe

# Convertir el diccionario dataframes_modificados en un DataFrame
df_nombres = pd.DataFrame(list(dataframes_modificados.items()), columns=['Nombre', 'DataFrame'])

# Guardar el DataFrame en un archivo Excel
df_nombres.to_excel('nombres_dataframes.xlsx', index=False)

# Confirmación de que se guardó el archivo
print("Se ha guardado el archivo 'nombres_dataframes.xlsx' con los nombres de los DataFrames.")



DataFrame 'Sony_PlayStation 4_playstation4':
                id           title  seller_id category_id     price  \
472  MLA1664900644  Play Station 4  231271055   MLA438566  360000.0   
595  MLA1590063836   Playstation 4  183558603   MLA438566  340000.0   
692  MLA1467337566   Playstation 4  200217743   MLA438566  360000.0   
709  MLA1412490311  Playstation 4   180607463   MLA438566  480000.0   
733  MLA1418522141   Playstation 4  291937267   MLA438566  590000.0   
817  MLA1627402592  Playstation 4    95023501   MLA438566  350000.0   
821  MLA1640369022   Playstation 4  290530220   MLA438566  550000.0   
833  MLA1412132291   Playstation 4  138164898   MLA438566  400000.0   
913  MLA1647383632  Play Station 4  205610083   MLA438566  450000.0   
917  MLA1630749836  Play Station 4  452958917   MLA438566  380000.0   
959  MLA1643211266  Play Station 4  442643421   MLA438566  450000.0   
988  MLA1716900152   Playstation 4   56577582   MLA438566  650000.0   

     base_price  original_price

In [34]:
# Obtener el número de DataFrames generados
numero_dataframes = len(dataframes_modificados)

# Mostrar el número de DataFrames generados
print("Número de DataFrames generados:", numero_dataframes)


Número de DataFrames generados: 35


In [35]:
# Crear un escritor de Excel
writer = pd.ExcelWriter('dataframes_resultantes.xlsx')

# Iterar sobre los DataFrames resultantes y guardar cada uno en una hoja de Excel
for nombre, dataframe in dataframes_modificados.items():
    # Truncar el nombre de la hoja si excede los 31 caracteres
    nombre_hoja = nombre[:31]
    
    # Eliminar columnas con todos los registros vacíos
    #dataframe = dataframe.dropna(axis=1, how='all')
    
    # Escribir el DataFrame en la hoja de Excel
    dataframe.to_excel(writer, sheet_name=nombre_hoja, index=False)

# Cerrar el escritor de Excel
writer.close()

# Confirmación de que se guardó el archivo
print("Se ha guardado el archivo 'dataframes_resultantes.xlsx' con cada DataFrame en una hoja separada.")

#aca tendriamos en un archivo excel 35 hojas(cada una es un datarame ) de items similares a partir de maseod e datos



Se ha guardado el archivo 'dataframes_resultantes.xlsx' con cada DataFrame en una hoja separada.
